# Simple Association Mining API Test

Clean, focused testing for your association rule mining server.

In [ ]:
# Import libraries
import requests
import json
import pandas as pd
from datetime import datetime

# Server configuration
BASE_URL = "http://localhost:8000"
API_BASE = f"{BASE_URL}/api/v1"

print(f"🌐 Testing server at: {BASE_URL}")
print(f"📅 Test time: {datetime.now()}")

In [ ]:
# 1. Health Check
print("🏥 Health Check")
print("-" * 30)

try:
    response = requests.get(f"{API_BASE}/health")
    if response.status_code == 200:
        print("✅ Server is healthy!")
        print(f"📊 Response: {response.json()}")
    else:
        print(f"❌ Health check failed: {response.status_code}")
except Exception as e:
    print(f"❌ Cannot connect to server: {e}")
    print("💡 Make sure server is running: py -m uvicorn app.main:app --reload")

In [ ]:
# 2. Start Mining
print("⛏️ Start Association Rule Mining")
print("-" * 30)

# Basic mining payload
payload = {
    "days_back": 30,
    "use_enhanced_mining": True,
    "time_weighting_method": "exponential_decay"
}

try:
    response = requests.post(f"{API_BASE}/mine-rules", json=payload)
    
    if response.status_code == 200:
        data = response.json()
        print("✅ Mining started successfully!")
        print(f"📝 Message: {data['message']}")
        print(f"🆔 Task ID: {data.get('task_id', 'N/A')}")
        
        # Store task_id for monitoring
        if 'task_id' in data:
            task_id = data['task_id']
            print(f"\n💾 Saved task_id: {task_id}")
    else:
        print(f"❌ Mining failed: {response.status_code}")
        print(f"📄 Error: {response.text}")
        
except Exception as e:
    print(f"❌ Error starting mining: {e}")

In [ ]:
# 3. Check Task Status
print("📊 Check Mining Status")
print("-" * 30)

try:
    # Check running tasks
    response = requests.get(f"{API_BASE}/tasks/running")
    
    if response.status_code == 200:
        data = response.json()
        running_count = data['count']
        
        if running_count == 0:
            print("✅ No tasks running - Mining completed!")
        else:
            print(f"🔄 {running_count} tasks still running:")
            for task in data['running_tasks']:
                print(f"   • {task['task_id'][:8]}... | {task['progress']:.1%} | {task['message'][:50]}...")
    else:
        print(f"❌ Cannot check status: {response.status_code}")
        
except Exception as e:
    print(f"❌ Error checking status: {e}")

# If you have a specific task_id, check its status
if 'task_id' in locals():
    print(f"\n🔍 Checking specific task: {task_id[:8]}...")
    try:
        response = requests.get(f"{API_BASE}/task/{task_id}")
        if response.status_code == 200:
            task_data = response.json()
            print(f"   Status: {task_data['status']}")
            print(f"   Progress: {task_data['progress']:.1%}")
            print(f"   Message: {task_data['message']}")
    except Exception as e:
        print(f"   ❌ Error: {e}")

In [ ]:
# 4. Get Recommendations
print("🎯 Test Recommendations")
print("-" * 30)

# Test items (modify these based on your data)
test_items = [
    "Coca Cola",
    "MAGGI 2-Minute Instant Noodles", 
    "Bread"
]

for item in test_items:
    print(f"\n🔍 Getting recommendations for: {item}")
    
    try:
        import urllib.parse
        encoded_item = urllib.parse.quote(item)
        response = requests.get(f"{API_BASE}/recommendations/{encoded_item}?limit=3")
        
        if response.status_code == 200:
            data = response.json()
            recommendations = data.get('recommendations', [])
            
            if recommendations:
                print(f"   ✅ Found {len(recommendations)} recommendations:")
                for i, rec in enumerate(recommendations, 1):
                    print(f"      {i}. {rec['recommended_item']} (Score: {rec['score']:.4f})")
            else:
                print("   ⚠️ No recommendations found")
        else:
            print(f"   ❌ Failed: {response.status_code} - {response.text}")
            
    except Exception as e:
        print(f"   ❌ Error: {e}")

In [ ]:
# 5. Export Rules to CSV
print("📁 Export Association Rules")
print("-" * 30)

try:
    # Get database connection info
    from app.utils.config import config
    import mysql.connector
    
    # Connect to database
    conn = mysql.connector.connect(
        host=config.DB_HOST,
        user=config.DB_USER,
        password=config.DB_PASSWORD,
        database=config.DB_NAME
    )
    
    # Query recommendations table
    query = f"""
    SELECT 
        main_item as sku1,
        recommended_item as sku2,
        composite_score as association_composite_score
    FROM {config.RECOMMENDATIONS_TABLE}
    ORDER BY composite_score DESC
    LIMIT 1000
    """
    
    df = pd.read_sql(query, conn)
    conn.close()
    
    if len(df) > 0:
        # Save to CSV
        filename = f"association_rules_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filename, index=False)
        
        print(f"✅ Exported {len(df)} rules to: {filename}")
        print(f"📊 Score range: {df['association_composite_score'].min():.4f} - {df['association_composite_score'].max():.4f}")
        
        # Show top 10 rules
        print("\n🏆 Top 10 Rules:")
        for i, (_, row) in enumerate(df.head(10).iterrows(), 1):
            print(f"   {i:2d}. {row['sku1'][:30]:<30} → {row['sku2'][:30]:<30} | {row['association_composite_score']:.4f}")
    else:
        print("⚠️ No rules found in database. Run mining first.")
        
except Exception as e:
    print(f"❌ Error exporting rules: {e}")
    print("💡 Make sure mining has completed and database is accessible")